In [1]:
from os import path, environ
from itertools import filterfalse
from functools import partial
from collections import namedtuple
import urllib.parse

from IPython.display import Image, display, HTML

import numpy as np
import pandas as pd

df = pd \
       .read_csv(path.join(path.expanduser('~'),
                           'OneDrive - The University of Sydney (Students)',
                           'DR SPOC - Graders 1 and 2.csv')) \
       .dropna(subset=('Record ID', 'Diagnosis assessment by ophthalmologist?')) \
       .set_index('Record ID')

# Made no difference:
#df.dropna(subset=tuple(column
#                       for column in df.columns
#                       if 'disc-centred photo' in column),
#          how='all', inplace=True)

display(HTML('<h2>Filtered DR SPOC - Graders 1 and 2</h2>'))
df

,Survey Timestamp,Gender,What ethnic background does the patient identify with?,Referral or data origin,For what suspected condition was the patient referred?,What type of diabetes does the patient have?,How long has the patient had diabetes (years)?,Comorbidities (choice=HTN),Comorbidities (choice=dyslipidaemia),Comorbidities (choice=ischaemic heart disease),...,Who has reviewed the fundus grading in the clinical record?,Management plan assessment,Diagnosis assessment by ophthalmologist?,Ophthal photo grading: Right eye ICDRSS,Ophthal photo grading: Right eye maculopathy,Ophthal photo grading: Left eye ICDRSS,Ophthal photo grading: Left eye maculopathy,Where was the suggested follow-up for this patient?.1,How soon was the suggested review of this patient?.1,Complete?.3
Record ID,,,,,,,,,,,,,,,,,,,,,
2,3/1/19 8:34,male,Other,Community Eye Care,diabetic retinopathy,Type 2,15.0,Unchecked,Unchecked,Unchecked,...,consultant ophthalmologist,Agree,Change,"Treated PDR, Stable",No diabetic maculopathy,"Treated PDR, Stable",No diabetic maculopathy,C-Eye-C,</= 6 months,Complete
4,3/1/19 10:27,female,Other,Community Eye Care,glaucoma,NaN,NaN,Unchecked,Unchecked,Unchecked,...,medical retina fellow,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,Complete
5,3/1/19 10:46,female,Other,Community Eye Care,glaucoma,NaN,NaN,Unchecked,Unchecked,Unchecked,...,medical retina fellow,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,Complete
6,3/1/19 11:09,male,Other,Community Eye Care,glaucoma,NaN,NaN,Unchecked,Unchecked,Unchecked,...,medical retina fellow,Change,Change,No DR,No diabetic maculopathy,No DR,No diabetic maculopathy,NaN,NaN,Incomplete
7,3/1/19 11:30,male,Aboriginal or Torres Strait Islander,Community Eye Care,glaucoma,NaN,NaN,Unchecked,Unchecked,Unchecked,...,medical retina fellow,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,Complete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,NaN,male,Other,Community Eye Care,diabetic retinopathy,Type 1,35.0,Unchecked,Unchecked,Unchecked,...,registrar,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,Complete
235,NaN,male,Other,Community Eye Care,diabetic retinopathy,Type 2,7.0,Unchecked,Unchecked,Unchecked,...,registrar,Change,Agree,No DR,No diabetic maculopathy,No DR,No diabetic maculopathy,Westmead Hospital Eye Clinic,</= 3 months,Complete
236,NaN,male,Other,Community Eye Care,diabetic retinopathy,Type 2,20.0,Unchecked,Unchecked,Unchecked,...,registrar,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,Complete


In [2]:
display(HTML('<h2>Disc-centred photo counts</h2>'))
df[filter(lambda column: 'disc-centred photo' in column,
          df.columns)] \
  .apply(pd.value_counts)

,Right disc-centred photo,Left disc-centred photo
yes,176,177
no,29,27


In [3]:
df[['What type of diabetes does the patient have?',
    'Who has performed the fundus grading in the clinical record?',
    'Diagnosis assessment by ophthalmologist?']] \
  .apply(pd.value_counts)

,What type of diabetes does the patient have?,Who has performed the fundus grading in the clinical record?,Diagnosis assessment by ophthalmologist?
Agree,NaN,NaN,169.0
Change,NaN,NaN,36.0
Type 1,13.0,NaN,NaN
Type 2,91.0,NaN,NaN
optometrist,NaN,204.0,NaN


In [4]:
display(HTML('<h2>Reviewer</h2>'))
df[['Who has reviewed the fundus grading in the clinical record?']] \
  .apply(pd.value_counts)

,Who has reviewed the fundus grading in the clinical record?
medical retina fellow,100
registrar,74
consultant ophthalmologist,30
no grading noted,1


In [5]:
display(HTML('<h2>Eye history</h2>'))
df[filter(lambda column: column.startswith('Eye history'), df.columns)] \
  .apply(pd.value_counts)

,Eye history (choice=cataract surgery),Eye history (choice=cataract),Eye history (choice=glaucoma),Eye history (choice=diabetic retinopathy),Eye history (choice=macular degeneration),Eye history (choice=eye trauma),Eye history (choice=family history of glaucoma),Eye history (choice=other)
Unchecked,182,185,195,204,203,200,179,189
Checked,23,20,10,1,2,5,26,16


In [6]:
display(HTML('<h2>Photo grading</h2>'))
df[filter(lambda column: column.startswith('Photo grading'), df.columns)] \
  .apply(pd.value_counts)

,Photo grading - diabetes: ICDRSS recorded for the right eye,Photo grading - diabetes: Maculopathy recorded for the right eye,Photo grading - diabetes: ICDRSS recorded for the left eye,Photo grading - diabetes: Maculopathy recorded for the left eye
HEx distant from fovea,NaN,2.0,NaN,NaN
"Maculopathy, unspecified",NaN,8.0,NaN,11.0
Mild NPDR,30.0,NaN,29.0,NaN
Mod NPDR,8.0,NaN,10.0,NaN
No DR,162.0,NaN,159.0,NaN
No diabetic maculopathy,NaN,193.0,NaN,193.0
No gradable image,1.0,1.0,1.0,1.0
Sev NPDR,1.0,NaN,2.0,NaN
"Treated PDR, Stable",2.0,NaN,3.0,NaN


In [7]:
display(HTML('<h2>Ophthal photo grading</h2>'))
df[filter(lambda column: column.startswith('Ophthal photo grading'), df.columns)] \
  .apply(pd.value_counts)

,Ophthal photo grading: Right eye ICDRSS,Ophthal photo grading: Right eye maculopathy,Ophthal photo grading: Left eye ICDRSS,Ophthal photo grading: Left eye maculopathy
"Maculopathy, unspecified",NaN,4.0,NaN,7.0
Mild NPDR,11.0,NaN,10.0,NaN
Mod NPDR,4.0,NaN,7.0,NaN
No DR,46.0,NaN,44.0,NaN
No diabetic maculopathy,NaN,61.0,NaN,59.0
Sev NPDR,1.0,NaN,1.0,NaN
"Treated PDR, Stable",3.0,NaN,3.0,NaN


In [8]:
display(HTML('<h2>Columns</h2>'))

df = df[
    filterfalse(
        lambda col: any((col.startswith('Eye'),
                         #not col.startswith('Ophthal photo') and
                         col.startswith('Comorbidities'),
                         #not col.startswith('Photo grading') and
                         col.startswith('Complete'),
                         'OCT' in col,
                         'Visual Acuity' in col,
                         'IOP' in col,
                         'Timestamp' in col,
                         #'recorded' in col
                        )),
        df.columns)]

display(HTML(
    '<ul>\n{}\n</ul>'.format('\n'.join(
        '  <li>"{}"</li>'.format(col)
        for col in sorted(df.columns)
))))

In [9]:
df[filter(lambda column: 'ICDRSS' in column or 'maculopathy' in column.lower(), df.columns)] \
  .apply(pd.value_counts)

,Photo grading - diabetes: ICDRSS recorded for the right eye,Photo grading - diabetes: Maculopathy recorded for the right eye,Photo grading - diabetes: ICDRSS recorded for the left eye,Photo grading - diabetes: Maculopathy recorded for the left eye,Ophthal photo grading: Right eye ICDRSS,Ophthal photo grading: Right eye maculopathy,Ophthal photo grading: Left eye ICDRSS,Ophthal photo grading: Left eye maculopathy
HEx distant from fovea,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
"Maculopathy, unspecified",NaN,8.0,NaN,11.0,NaN,4.0,NaN,7.0
Mild NPDR,30.0,NaN,29.0,NaN,11.0,NaN,10.0,NaN
Mod NPDR,8.0,NaN,10.0,NaN,4.0,NaN,7.0,NaN
No DR,162.0,NaN,159.0,NaN,46.0,NaN,44.0,NaN
No diabetic maculopathy,NaN,193.0,NaN,193.0,NaN,61.0,NaN,59.0
No gradable image,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
Sev NPDR,1.0,NaN,2.0,NaN,1.0,NaN,1.0,NaN
"Treated PDR, Stable",2.0,NaN,3.0,NaN,3.0,NaN,3.0,NaN


- no DR & no diabetic maculopathy would classify as 'healthy'
- no gradable image would classify as 'ungradable' and 
- everything else would be 'DR'

In [10]:
df['Photo grading - diabetes: Maculopathy recorded for the right eye'].unique()

array(['No diabetic maculopathy', 'Maculopathy, unspecified',
       'No gradable image', 'HEx distant from fovea', nan], dtype=object)

In [11]:
df[filter(lambda column: 'ICDRSS' in column, df.columns)] \
  .apply(pd.value_counts)

,Photo grading - diabetes: ICDRSS recorded for the right eye,Photo grading - diabetes: ICDRSS recorded for the left eye,Ophthal photo grading: Right eye ICDRSS,Ophthal photo grading: Left eye ICDRSS
Mild NPDR,30,29,11.0,10.0
Mod NPDR,8,10,4.0,7.0
No DR,162,159,46.0,44.0
No gradable image,1,1,NaN,NaN
Sev NPDR,1,2,1.0,1.0
"Treated PDR, Stable",2,3,3.0,3.0


In [12]:
df[filter(lambda column: column.endswith('eye maculopathy'), df.columns)] \
  .apply(pd.value_counts)

,Ophthal photo grading: Right eye maculopathy,Ophthal photo grading: Left eye maculopathy
No diabetic maculopathy,61,59
"Maculopathy, unspecified",4,7


In [13]:
df[filter(lambda column: 'hoto' in column, df.columns)].columns

Index(['Right macular-centred photo', 'Right disc-centred photo',
       'Left macular-centred photo', 'Left disc-centred photo',
       'Photo grading - diabetes: ICDRSS recorded for the right eye',
       'Photo grading - diabetes: Maculopathy recorded for the right eye',
       'Photo grading - diabetes: ICDRSS recorded for the left eye',
       'Photo grading - diabetes: Maculopathy recorded for the left eye',
       'Ophthal photo grading: Right eye ICDRSS',
       'Ophthal photo grading: Right eye maculopathy',
       'Ophthal photo grading: Left eye ICDRSS',
       'Ophthal photo grading: Left eye maculopathy'],
      dtype='object')

In [14]:
def upsert(obj, column, value):
    def _upsert():
        is_list = isinstance(obj.columns[column], list)
        if isinstance(obj.columns[column], tuple):
            obj.columns[column] = list(obj.columns[column]) + [value]
        elif len(obj.columns[column]) == 0:
            obj.columns[column] = [value]
        elif not is_list:
            obj.columns[column] = [obj.columns[column], value]
        elif is_list:
            obj.columns[column].append(obj.columns[column])
        else:
            raise TypeError('{} not supported (found at column: {!r} with value: {!r})'.format(
                type(obj), column, value
            ))
        return obj

    if hasattr(obj, 'columns'):
        if column in obj.columns:
            return _upsert()
    elif hasattr(obj, '__contains__'):
        if column in obj:
            return _upsert()
    elif not hasattr('__setattr__'):
        raise TypeError('{} not supported (found at column: {!r} with value: {!r})'.format(
            type(obj), column, value
        ))

    obj[column] = [value]

    return _upsert()

In [15]:
def maybe_rename_column(column):
    suffix = 'right' if 'Right' in column else 'left'
    columns = 'ICDRSS', 'maculopathy'
    for col in columns:
        if column.endswith(col):
            return '_'.join((suffix, col))
        elif '{col} recorded'.format(col=col).lower() in column.lower():
            return '_'.join((suffix, col, 'photo'))
        else:
            pass
            #display('"{col} recorded" in "{column}" gave {r}'.format(col=col, column=column.lower(),
            #                                          r='{col} recorded'.format(col=col) in column.lower()))
    return column

In [16]:
'''
elif np.array_equal(
    np.array(['Mod NPDR',
              'No DR',
              'Mild NPDR',
              'Treated PDR, Stable',
              'No gradable image', # np.nan,
              'Sev NPDR'], dtype=object),
    found_cols):

    return get_sum_df({
        'referrable': ['Mod NPDR', 'Mild NPDR', 'Treated PDR, Stable', 'Sev NPDR'],
        'non-referable': ['No DR'],
        'ungradable': ['No gradable image']
    })

# maculopathy
elif np.array_equal(
    np.array(['No diabetic maculopathy', #np.nan,
              'Maculopathy, unspecified'],
             dtype=object),
    found_cols):
    return get_sum_df({
        'referrable': ['Maculopathy, unspecified'],
        'non-referable': ['No diabetic maculopathy']
    })

elif np.array_equal(
    np.array(['No diabetic maculopathy', 'Maculopathy, unspecified',
              'No gradable image', 'HEx distant from fovea' # , np.nan
             ], dtype=object),
    found_cols):

    return get_sum_df({
        'referrable': ['Maculopathy, unspecified', 'HEx distant from fovea' ],
        'non-referable': ['No diabetic maculopathy'],
        'ungradable': ['No gradable image']
    })
    '''
pass

In [17]:
'''
elif np.array_equal(
    np.array(['Mod NPDR', 'No DR', 'Mild NPDR', 'Treated PDR, Stable',
              'No gradable image', #np.nan,
              'Sev NPDR'], dtype=object),
    found_cols):            
    def ICDRSS_to_3_cat(column):
        if pd.isnull(column):
            return
        return {'Mod NPDR': 'referable',
                'No DR': 'non-referable',
                'Mild NPDR': 'referable',
                'Treated PDR, Stable': 'referable',
                'No gradable image': 'ungradable', #np.nan,
                'Sev NPDR': 'referable'}[column]

    return series.apply(ICDRSS_to_3_cat).to_frame()

# maculopathy
elif np.array_equal(
    np.array(['No diabetic maculopathy', #np.nan,
              'Maculopathy, unspecified'],
             dtype=object),
    found_cols):
    def maculopathy_to_3_cat(column):
        # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
        if pd.isnull(column):
            return
        return {'No diabetic maculopathy': 'non-referable',
                'Maculopathy, unspecified': 'non-referable'}[column]

    return series.apply(maculopathy_to_3_cat).to_frame()
elif np.array_equal(
    np.array(['No diabetic maculopathy', 'Maculopathy, unspecified',
              'No gradable image', 'HEx distant from fovea' # , np.nan
             ], dtype=object),
    found_cols):
    def maculopathy_to_3_cat(column):
        # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
        if pd.isnull(column):
            return
        return {'No diabetic maculopathy': 'non-referable',
                'Maculopathy, unspecified': 'referable',
                'No gradable image': 'ungradable',
                'HEx distant from fovea': 'referable'}[column]

    return series.apply(maculopathy_to_3_cat).to_frame()
'''
pass

In [18]:
def to_3_cat(series):
    assert isinstance(series, pd.Series), 'Expected `pd.Series` got `{!r}`'.format(type(series))
    
    series_value_counts = pd.DataFrame(series) \
                            .reset_index() \
                            .rename(index={'Record ID': 'id'}) \
                            .drop('Record ID', axis=1) \
                            .apply(pd.value_counts)
    
    def get_sum_df(d):
        return pd.DataFrame.from_records([{k: series_value_counts.loc[v].sum().values[0]
                                           for k, v in d.items()}],
                                        index=[maybe_rename_column(series_value_counts.columns[0])])
    
    found_cols = (lambda a: a[~pd.isnull(a)])(series.unique())
    series.dropna(inplace=True)

    # ICDRSS
    if np.array_equal(
        np.array(['Treated PDR, Stable', # np.nan,
                  'No DR', 'Mild NPDR', 'Mod NPDR', 'Sev NPDR'], dtype=object),
        found_cols):
        
        
        def ICDRSS_to_3_cat(column): 
            if pd.isnull(column):
                return
            return {'Treated PDR, Stable': 'referable',
                    'No DR': 'non-referable',
                    'Mild NPDR': 'referable',
                    'Mod NPDR': 'referable',
                    'Sev NPDR': 'referable'}[column]
        
        return series.apply(ICDRSS_to_3_cat).to_frame()
    elif np.array_equal(
        np.array(['Mod NPDR', 'No DR', 'Mild NPDR', 'Treated PDR, Stable',
                  'No gradable image', #np.nan,
                  'Sev NPDR'], dtype=object),
        found_cols):            
        def ICDRSS_to_3_cat(column):
            if pd.isnull(column):
                return
            return {'Mod NPDR': 'referable',
                    'No DR': 'non-referable',
                    'Mild NPDR': 'referable',
                    'Treated PDR, Stable': 'referable',
                    'No gradable image': 'ungradable', #np.nan,
                    'Sev NPDR': 'referable'}[column]

        return series.apply(ICDRSS_to_3_cat).to_frame()

    # maculopathy
    elif np.array_equal(
        np.array(['No diabetic maculopathy', #np.nan,
                  'Maculopathy, unspecified'],
                 dtype=object),
        found_cols):
        def maculopathy_to_3_cat(column):
            # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
            if pd.isnull(column):
                return
            return {'No diabetic maculopathy': 'non-referable',
                    'Maculopathy, unspecified': 'non-referable'}[column]
        
        return series.apply(maculopathy_to_3_cat).to_frame()
    elif np.array_equal(
        np.array(['No diabetic maculopathy', 'Maculopathy, unspecified',
                  'No gradable image', 'HEx distant from fovea' # , np.nan
                 ], dtype=object),
        found_cols):
        def maculopathy_to_3_cat(column):
            # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
            if pd.isnull(column):
                return
            return {'No diabetic maculopathy': 'non-referable',
                    'Maculopathy, unspecified': 'referable',
                    'No gradable image': 'ungradable',
                    'HEx distant from fovea': 'referable'}[column]
        
        return series.apply(maculopathy_to_3_cat).to_frame()
    elif np.array_equal(
        np.array(['No diabetic maculopathy', 'Maculopathy, unspecified', 'No gradable image'], dtype=object),
        found_cols):
        def maculopathy_to_3_cat(column):
            # display(HTML('<h4>maculopathy::column: {}\t{}</h4>'.format(column, pd.isnull(column))))
            if pd.isnull(column):
                return
            return {'No diabetic maculopathy': 'non-referable',
                    'Maculopathy, unspecified': 'referable',
                    'No gradable image': 'ungradable'}[column]
        
        return series.apply(maculopathy_to_3_cat).to_frame()        
    else:
        display(HTML('No match for {} DONE with {!r}'.format(
            series.to_frame()[:3].to_html(),
            series.unique()
        )))

In [19]:
#selected_columns = filter(lambda column: 'hoto grading' in column or 'recorded' in column, df.columns)
#selected_columns = 'Ophthal photo grading: Right eye ICDRSS',
selected_columns = filter(
    lambda column: column.startswith('Photo grading') or column.startswith('Ophthal photo grading'),
    df.columns
)
cat3_df = pd.concat((to_3_cat(df[column].copy())
                     for column in selected_columns), sort=True)
cat3_df.apply(pd.value_counts)

,Ophthal photo grading: Left eye ICDRSS,Ophthal photo grading: Left eye maculopathy,Ophthal photo grading: Right eye ICDRSS,Ophthal photo grading: Right eye maculopathy,Photo grading - diabetes: ICDRSS recorded for the left eye,Photo grading - diabetes: ICDRSS recorded for the right eye,Photo grading - diabetes: Maculopathy recorded for the left eye,Photo grading - diabetes: Maculopathy recorded for the right eye
non-referable,44.0,66.0,46.0,65.0,159,162,193,193
referable,21.0,NaN,19.0,NaN,44,41,11,10
ungradable,NaN,NaN,NaN,NaN,1,1,1,1


In [20]:
LocationPid = namedtuple('LocationPid', ('location', 'pid'))

def parseFname(fname):
    bname = path.splitext(path.basename(fname))[0]
    buffer, location, pid, last_char = '', '', '', ''
    i = 0
    while i < len(bname):
        location = last_char + bname[i]
        if location in frozenset(('R1', 'R2', 'L1', 'L2')):
            pid = pid[:-1]
            break
        pid += bname[i]
        last_char = bname[i]
        i += 1
    return LocationPid(location=location, pid=pid)




def sql_gen(fname, diagnosis):
    location, pid = parseFname(fname)
    prefix = 'fundus_images/'
    quoted_location = urllib.parse.quote(
        '{prefix}DR SPOC Photo Dataset/6146/Upload/WA112325R2-4.jpg'.format(prefix=prefix),
        safe=''
    )
    return '\n'.join(map(lambda l: l.lstrip(' '), '''
        BEGIN TRANSACTION;

        INSERT INTO artifact_tbl (location, "contentType")
        VALUES ('{quoted_location}', 'image/jpeg');

        INSERT INTO categorise_tbl ("artifactLocation", "categoryEnumName", category, username)
        VALUES ('{quoted_location}',
                \t'Simple-categories', '{diagnosis}', 'spreadsheet');

        END TRANSACTION;
        '''.format(quoted_location=quoted_location, diagnosis=diagnosis).split('\n')))

print(sql_gen(#(
    'DR SPOC Photo Dataset/6146/Upload/WA112325R2-8.jpg', 'REFERABLE')
    #('DR SPOC Photo Dataset/6146/Upload/WA112325R2-9.jpg', 'NON-REFERABLE'),
    #('DR SPOC Photo Dataset/6146/Upload/WA112325R2-1.jpg', 'UNGRADABLE')
#))
)


BEGIN TRANSACTION;

INSERT INTO artifact_tbl (location, "contentType")
VALUES ('fundus_images%2FDR%20SPOC%20Photo%20Dataset%2F6146%2FUpload%2FWA112325R2-4.jpg', 'image/jpeg');

INSERT INTO categorise_tbl ("artifactLocation", "categoryEnumName", category, username)
VALUES ('fundus_images%2FDR%20SPOC%20Photo%20Dataset%2F6146%2FUpload%2FWA112325R2-4.jpg',
	'Simple-categories', 'REFERABLE', 'spreadsheet');

END TRANSACTION;



In [21]:
import psycopg2

result = urllib.parse.urlparse(environ['RDBMS_URI'])

with psycopg2.connect(
    database = result.path[1:],
    user = result.username,
    password = result.password,
    host = result.hostname
) as conn:
    with conn.cursor() as curs:
        curs.execute('SELECT 5*5')
        print(curs.fetchone())

(25,)
